In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import sys                                                                      #
sys.path.append('/content/drive/My Drive/nn_method')                                  #
!pip install wikipedia                                                          #
!pip install unidecode                                                          #
import numpy as np                                                              #
from keras.models import Sequential                                             #
from keras.layers import Dense                                                  #
from keras.layers import Embedding                                              #             Importing necessary packages
from keras.callbacks import ModelCheckpoint, TensorBoard                        #
from sklearn.model_selection import train_test_split                            #
from config import language_tags, max_letters                                   #
import random                                                                   #
from itertools import chain                                                     #
from functions import convert_dic_to_vector , process                           #

In [0]:
data = np.load('/content/drive/My Drive/nn_method/arr.npy','r')
# print(len(language_tags))
inputs = data[:, 4:]                                                 #       splitting data into inputs and labels
labels = data[:, 1:4]                                                #       
                                                                                        #
x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.15)     #       splitting data into into test and validation sets

In [6]:
network = Sequential()                                                                  #
network.add(Dense(200, input_dim=26*max_letters, activation='relu'))                    #
network.add(Dense(150, activation='relu'))                                              #
network.add(Dense(100, activation='relu'))                                              #           Defining Model
network.add(Dense(100, activation='relu'))                                              #
network.add(Dense(3, activation='softmax'))                            #
                                                                                        #
network.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])     #






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
filepath = "weights.hdf5"                                                                                   #   
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')       #
tboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)                               #           Model fitting
callbacks_list = [checkpoint, tboard]                                                                     #
                                                                                                          #
network.fit(x_train, y_train, epochs=50, batch_size=1000, validation_data=(x_test, y_test), callbacks=callbacks_list)   #




Train on 32471 samples, validate on 5731 samples







Epoch 1/50
32471/32471 [==============================] - 11s 337us/step - loss: 0.5309 - acc: 0.7318 - val_loss: 0.3553 - val_acc: 0.8487

Epoch 00001: val_acc improved from -inf to 0.84866, saving model to weights.hdf5

Epoch 2/50
32471/32471 [==============================] - 10s 305us/step - loss: 0.2972 - acc: 0.8787 - val_loss: 0.2486 - val_acc: 0.8971

Epoch 00002: val_acc improved from 0.84866 to 0.89711, saving model to weights.hdf5
Epoch 3/50
32471/32471 [==============================] - 10s 307us/step - loss: 0.2024 - acc: 0.9214 - val_loss: 0.1998 - val_acc: 0.9168

Epoch 00003: val_acc improved from 0.89711 to 0.91683, saving model to weights.hdf5
Epoch 4/50
32471/32471 [==============================] - 10s 305us/step - loss: 0.1541 - acc: 0.9402 - val_loss: 0.1736 - val_acc: 0.9331

Epoch 00004: val_acc improved from 0.91683 to 0.93305, saving model to weights.hdf5
Epoch 5/50
32471/32471 [=======================

In [8]:
file1 = open('/content/drive/My Drive/nn_method/test.txt', 'r')                       # 
Lines = file1.readlines()                                                       #
punctuations = ".,!?;'/\n"                                                      #   Opening test file. 
no_punct = ""                                                                   #   Model reports the language of this test file
alpha=[]                                                                        #

for line in Lines:                                                              #
    temp=line.lower()                                                           #
    for char in temp:                                                           #   
        if char in punctuations:                                                #     Processing test.txt file
            temp=temp.replace(char,"")                                          #
    alpha.append(temp)                                                          #

alpha1=[]                                                                       #
for j in range (len(alpha)):                                                    #
  alpha1.append(process(alpha[j],200))                                          #


alpha1 = list(chain.from_iterable(alpha1))  
print(alpha1)                                                                   #   printing words obtained

network = Sequential()                                                          #
network.add(Dense(200, input_dim=26*max_letters, activation='relu'))            #
network.add(Dense(150, activation='relu'))                                      #
network.add(Dense(100, activation='relu'))                                      #       Loading pre-trained model with weights
network.add(Dense(100, activation='relu'))                                      #
network.add(Dense(3, activation='softmax'))                                     #
network.load_weights('weights.hdf5')                                            #
network.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy']) #

temp0=np.zeros((1,3))                                                           #

dic = []                                                                        #         
for word in alpha1 :                                                            #
  dic.append(word)                                                              #   Converting the processed string to vector
  vct_str = convert_dic_to_vector(dic, max_letters)                             #  
  vct = np.zeros((1, 26 * max_letters))                                         #
  count = 0                                                                       #


  for digit in vct_str[0]:                                                        #
      vct[0,count] = int(digit)                                                 #
      count += 1                                                                #
  prediction_vct = network.predict(vct)                                         #    
  temp0+=prediction_vct                                                         #     Summing individual probabilities for each words
                                                                                

t=np.argmax(temp0[0])                                                           #   

if t==0:                                                                        #
  print("English")                                                              #
if t==1 :                                                                       #
  print("German")                                                               #       Reporting Language with cumulative highest probability
if t==2 :                                                                       #
  print("French")                                                               #                                                                   #

['poems', 'awaken', 'the', 'dormant', 'soul', 'in', 'us', 'we', 'all', 'come', 'at', 'place', 'where', 'we', 'feel', 'dejected', 'and', 'disenfranchised', 'with', 'this', 'materialistic', 'world', 'once', 'in', 'our', 'lives', 'at', 'that', 'time', 'of', 'undetermined', 'condition', 'and', 'confusion', 'the', 'poems', 'will', 'make', 'our', 'heart', 'clear', 'of', 'all', 'the', 'debris', 'and', 'unwanted', 'feelings', 'and', 'make', 'our', 'heart', 'pure', 'with', 'the', 'awakened', 'soul', 'we', 'have', 'to', 'make', 'it', 'clear', 'that', 'we', 'want', 'peace', 'and', 'love', 'in', 'this', 'wonderful', 'life', 'over', 'all', 'other', 'things', 'of', 'materialism', 'here', 'are', 'those', 'poems', 'which', 'come', 'under', 'this', 'kind', 'of', 'nature', 'to', 'make', 'yourself', 'proud', 'of', 'having', 'been', 'born', 'man', 'woman', 'in', 'this', 'beautiful', 'world', 'we', 'have', 'collected', 'these', 'poems', 'with', 'deligence', 'and', 'longwaiting', 'the', 'list', 'may', 'vary